In [1]:
import os
import time
import math
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
chain = Net.POLYGON
platform = Platform.SUSHI
contract = JSONContract.UniswapV2Pair

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [3]:
#dir(Pair)

In [4]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoading().get_contract(web3, abi_path)
mint_filt = Filter.create_filter(address=None, event_types=[Pair.events.Mint])  # Listen events from any smart contract

In [5]:
reorg_mon = JSONRPCReorganizationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [6]:
start = start - 100

In [7]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=mint_filt)

In [8]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    topics = evt["topics"]
    arguments = Conversion().decode_data(evt["data"])

    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt["event"]
    event['details']['buyer'] = Conversion().convert_uint256_hex_string_to_address(topics[1])

    if(len(arguments) >= 1):
        event['details']['purchase_price'] = Conversion().convert_int256_bytes_to_int(arguments[0])
    else:
        event['details']['transfer_value'] = math.nan
     
    events[k] = event
    if key not in processed_events:
        print(f"Transfer at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Transfer at block:60,957,945 tx:0xea739c89202c869fac0c3db381d7c76b522a57c73a91e8b925caae9e95be4fb5
Transfer at block:60,957,947 tx:0xe88ae63fd390969e2393a791be8525e4ca29d4c78d72078728fa78c3cad1a098
Transfer at block:60,957,948 tx:0x8da697c8c546e07bac41f17494c309e79b6458e55ce7bc5adeaffb6be03da02c
Transfer at block:60,957,955 tx:0x33dd7ae119557e83d5e77138a42d67bbad76ca6c6c74bc7ed8fd2ec2d1b01c5d
Transfer at block:60,957,955 tx:0x33dd7ae119557e83d5e77138a42d67bbad76ca6c6c74bc7ed8fd2ec2d1b01c5d
Transfer at block:60,957,955 tx:0x33dd7ae119557e83d5e77138a42d67bbad76ca6c6c74bc7ed8fd2ec2d1b01c5d
Transfer at block:60,957,955 tx:0x33dd7ae119557e83d5e77138a42d67bbad76ca6c6c74bc7ed8fd2ec2d1b01c5d
Transfer at block:60,957,955 tx:0x81fe4f4398aeea899cfdfa5a68b0f43f13656fe3132bf5b5bf97cf151321a8b7
Transfer at block:60,957,955 tx:0x81fe4f4398aeea899cfdfa5a68b0f43f13656fe3132bf5b5bf97cf151321a8b7
Transfer at block:60,957,959 tx:0x36ae0ba5701a23878dec5324945983b7a3503e85907b6011e980c1e00ceaeca9
Transfer a

In [9]:
events

{0: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'mint',
  'platform': 'sushi',
  'address': '0x41171356e89ecc68e2b468636e63667d396bff0f',
  'tx_hash': '0xea739c89202c869fac0c3db381d7c76b522a57c73a91e8b925caae9e95be4fb5',
  'blk_num': 60957945,
  'timestamp': 1724451861,
  'details': {'web3_type': web3._utils.datatypes.Mint,
   'buyer': '0xe8407A63f54BF0E848b00492A046AaC09fa49670',
   'purchase_price': 1724451861}},
 1: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'mint',
  'platform': 'sushi',
  'address': '0x60d55f02a771d515e077c9c2403a1ef324885cec',
  'tx_hash': '0xe88ae63fd390969e2393a791be8525e4ca29d4c78d72078728fa78c3cad1a098',
  'blk_num': 60957947,
  'timestamp': 1724451865,
  'details': {'web3_type': web3._utils.datatypes.Mint,
   'buyer': '0xe8599F3cc5D38a9aD6F3684cd5CEa72f10Dbc383',
   'purchase_price': 3022421}},
 2: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'mint',
  'platform': 'sushi',
  'address': '0x41171356e89ecc68

In [10]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv2pair,mint,sushi,0x41171356e89ecc68e2b468636e63667d396bff0f,0xea739c89202c869fac0c3db381d7c76b522a57c73a91...,60957945,1724451861,{'web3_type': <class 'web3._utils.datatypes.Mi...
1,polygon,uniswapv2pair,mint,sushi,0x60d55f02a771d515e077c9c2403a1ef324885cec,0xe88ae63fd390969e2393a791be8525e4ca29d4c78d72...,60957947,1724451865,{'web3_type': <class 'web3._utils.datatypes.Mi...
2,polygon,uniswapv2pair,mint,sushi,0x41171356e89ecc68e2b468636e63667d396bff0f,0x8da697c8c546e07bac41f17494c309e79b6458e55ce7...,60957948,1724451867,{'web3_type': <class 'web3._utils.datatypes.Mi...
3,polygon,uniswapv2pair,mint,sushi,0x1a13f4ca1d028320a707d99520abfefca3998b7f,0x33dd7ae119557e83d5e77138a42d67bbad76ca6c6c74...,60957955,1724451883,{'web3_type': <class 'web3._utils.datatypes.Mi...
4,polygon,uniswapv2pair,mint,sushi,0x1a13f4ca1d028320a707d99520abfefca3998b7f,0x33dd7ae119557e83d5e77138a42d67bbad76ca6c6c74...,60957955,1724451883,{'web3_type': <class 'web3._utils.datatypes.Mi...
5,polygon,uniswapv2pair,mint,sushi,0x28424507fefb6f7f8e9d3860f56504e4e5f5f390,0x33dd7ae119557e83d5e77138a42d67bbad76ca6c6c74...,60957955,1724451883,{'web3_type': <class 'web3._utils.datatypes.Mi...
6,polygon,uniswapv2pair,mint,sushi,0x1a13f4ca1d028320a707d99520abfefca3998b7f,0x33dd7ae119557e83d5e77138a42d67bbad76ca6c6c74...,60957955,1724451883,{'web3_type': <class 'web3._utils.datatypes.Mi...
7,polygon,uniswapv2pair,mint,sushi,0x1a13f4ca1d028320a707d99520abfefca3998b7f,0x81fe4f4398aeea899cfdfa5a68b0f43f13656fe3132b...,60957955,1724451883,{'web3_type': <class 'web3._utils.datatypes.Mi...
8,polygon,uniswapv2pair,mint,sushi,0x1a13f4ca1d028320a707d99520abfefca3998b7f,0x81fe4f4398aeea899cfdfa5a68b0f43f13656fe3132b...,60957955,1724451883,{'web3_type': <class 'web3._utils.datatypes.Mi...
9,polygon,uniswapv2pair,mint,sushi,0x41171356e89ecc68e2b468636e63667d396bff0f,0x36ae0ba5701a23878dec5324945983b7a3503e85907b...,60957959,1724451891,{'web3_type': <class 'web3._utils.datatypes.Mi...
